


# HW 5 - Building a normalized RDB

The goal of this homework is to take a semi-structured non-normalized CSV file and turn it into a set of normalized tables that you then push to your MySQL database on AWS (or to your local MySQL).

The original dataset contains 100k district court decisions, but it has been to downsampled to only 1000 rows to make the uploads faster. Each row contains info about a judge, their demographics, party affiliation, etc. Rows also contain information about the case they were deciding on. Was it a criminal or civil case? What year was it? Was the direction of the decision liberal or conservative?

While the current denormalized format is fine for analysis, it's not fine for a database as it violates many normalization rules. Your goal is to normalize it by designing a simple schema, then wrangling it into the proper dataframes, then pushing it all to database server.

For the first part of this assignment you should wind up with four tables. One with case information, one with judge information, one that has casetype information, and for category info. Each table should be reduced so that there are not then repeating rows, and primary keys should be assigned within each. These tables should be called 'casedb_case', 'casedb_judge','casedb_casetype', and 'casedb_category'.

For the last part you should make a rollup table that calculates the percent of liberal decisions for each party level and each case category. This will allow for one to get a quick look at how the political party affiliation of judges impacts the direction of a decision for different case categories (e.g. criminal, civil, labor).

**Submission**

1) Make a copy and replace blank with your name

2) Complete and run all cells. (For DDL and DML cells, re-running will result in error unless you drop your table first)

3) Download .ipynb of the notebook (make sure all cells have appropariate output).

4) Submit on Gradescope


## Bring in data, explore, make schema

Start by bringing in your data to `cases`. Call a `.head()` on it to see what columns are there and what they contain.

In [63]:
import pandas as pd
all_cases_df = pd.read_csv('https://docs.google.com/spreadsheets/d/1AWLK06JOlSKImgoHNTbj7oXR5mRfsL2WWeQF6ofMq1g/gviz/tq?tqx=out:csv')

In [64]:
all_cases_df.head()

,judge_name,party_name,gender_name,race_name,case_id,case_year,casetype_id,casetype_name,category_id,category_name,libcon_id,libcon_name
0,"Thompson, Myron H.",Democrat,male,African-American/black,28321332,2011,3,criminal court motions,1,Criminal Justice Cases,0,Conservative
1,"Shoob, Marvin A.",Democrat,male,white/caucasian,18110669,1993,14,free of religion,2,Civil Liberties/Rights Cases,1,Liberal
2,"Bua, Nicholas J.",Democrat,male,white/caucasian,15660871,1983,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
3,"Kovachevich, Elizabeth",Republican,female,white/caucasian,17770934,1991,2,habeas corpus-state,1,Criminal Justice Cases,0,Conservative
4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian,26621195,2009,6,alien petitions,2,Civil Liberties/Rights Cases,0,Conservative


### Make schema

OK, given that head, you need to make four related tables that will make up a normalized database. Those tables are 'casedb_cases', 'casedb_judges', 'casedb_category', and 'casedb_casetype'. If it's not clear what info should go into each, explore the data more. Find the functional dependencies, and create the tables based on thoes.

Remember, you might not have keys, will need to reduce the rows, select certain columns, etc. There isn't a defined path here.



Let's start by bringing the coonection info, run_query, and sql_head

In [65]:
!pip install mysql-connector-python

In [66]:
import mysql.connector

In [67]:
#get_conn_cur/run_query/sql_head
mysql_address  = 'my-dataengineering-db.cfkwimcau8wa.us-east-2.rds.amazonaws.com'
mysql_username='admin'
mysql_password='Catsrule99'
mysql_database = 'my_dataengineering_db'

def get_conn_cur():
    cnx = mysql.connector.connect(user=mysql_username, password=mysql_password,
          host=mysql_address,
          database=mysql_database, port='3306');
    return (cnx, cnx.cursor())

def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor

  cur.execute(query_string) # executing string as before

  my_data = cur.fetchall() # fetch query data as before

  result_df = pd.DataFrame(my_data, columns=cur.column_names)


  cur.close() # close
  conn.close() # close

  return result_df

def sql_head(table_name: str):
    return run_query(f"SELECT * FROM {table_name} LIMIT 5")

## Make casetype - 5 points


We start by tables that do not have foreign keys. First create a table that contains just each casetype info. I would call this table that you're going to upload `casestype_df` so you don't overwrite your raw data.

Go make the casetype table. This should have only two columns that allow you to link the casetype name back to the ID in the 'cases' table. Note that when you select attributes from the `all_cases_df` there would be many duplicated rows, so you have remove duplicated rows using `drop_dublicates`. Finally, there should be only 27 rows for casetype.



In [68]:
# Make casetype_df
casetype_df = all_cases_df[['casetype_id','casetype_name']].drop_duplicates()


### Make cases table in your database

Put the helper function to create the connection here.
Once you do that you'll need to do the following

* Connect, make a table called 'casedb_casetype' with the correct column names,data types, and primary key. Be sure to execute and commit the table.
* Make tuples of your data
* Write a SQL string that allows you to insert each tuple of data into the correct columns
* Execute the string many times to fill out 'cases'
* Commit changes and check the table.

I'm not going to leave a full roadmap beyond this. Feel free to add cells as needed to do the above.

In [69]:
#create casedb_casetype table
run_query("""
  CREATE TABLE casedb_casetype (
    casetype_id INT NOT NULL,
    casetype_name VARCHAR(255) NOT NULL,
    PRIMARY KEY (casetype_id)
  );
""")

ProgrammingError: 1050 (42S01): Table 'casedb_casetype' already exists

In [70]:
#run this cell
sql_head(table_name='casedb_casetype')

,casetype_id,casetype_name
0,1,habeas corpus-US
1,2,habeas corpus-state
2,3,criminal court motions
3,4,contempt of court
4,5,(non)conv-criminal case


In [71]:
#load data into casedb_casetype
import numpy as np
data_tups = [tuple(x) for x in casetype_df.to_numpy()]
iq = "INSERT INTO casedb_casetype (casetype_id, casetype_name) VALUES (%s, %s)"
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

IntegrityError: 1062 (23000): Duplicate entry '3' for key 'casedb_casetype.PRIMARY'

In [72]:
#TEST #this must return 27
run_query("""SELECT COUNT(*) FROM casedb_casetype;""")

,COUNT(*)
0,27


In [73]:
#TEST #this must return contempt of court
run_query("""SELECT casetype_name FROM casedb_casetype WHERE casetype_id = 4;""")

,casetype_name
0,contempt of court


#Make Categoty 5 points

Do the same to create the `casedb_category` table and load data

In [74]:
#create category_df
category_df = all_cases_df[['category_id','category_name']].drop_duplicates()


In [75]:
#create table
run_query("""
  CREATE TABLE casedb_category (
    category_id INT NOT NULL,
    category_name VARCHAR(255) NOT NULL,
    PRIMARY KEY (category_id)
  );
""")

ProgrammingError: 1050 (42S01): Table 'casedb_category' already exists

In [76]:
#load data
data_tups = [tuple(x) for x in category_df.to_numpy()]
iq = "INSERT INTO casedb_category (category_id, category_name) VALUES (%s, %s)"
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

IntegrityError: 1062 (23000): Duplicate entry '1' for key 'casedb_category.PRIMARY'

In [77]:
#TEST
run_query("SELECT COUNT(*) FROM casedb_category;")

,COUNT(*)
0,3


In [78]:
#TEST
run_query("SELECT category_name FROM casedb_category WHERE category_id = 3;")

,category_name
0,Labor & Economic Cases


In [ ]:
#[-2] each failed test
#[-1] missing primary key
#[-4] only create dataframe

## Make judges - 5 points

Now make your judges table from the original `all_cases_df` dataframe.

Judges should have five columns, including the `judge_id` that you have to create using `pd.factorize` (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html) on judge name. There should be 553 rows after you drop duplicates (remember that judges may have had more than one case).

After you make the dataset go and push to a SQL table called 'judges'.

In [79]:
#Create judge id and assign to judge_id attribute on all_cases_df
judge_id = pd.factorize(all_cases_df['judge_name'])[0]
all_cases_df['judge_id'] = judge_id

In [80]:
#create judge_df
judge_df = all_cases_df[['judge_id','judge_name','party_name','gender_name','race_name']].drop_duplicates()
judge_df

,judge_id,judge_name,party_name,gender_name,race_name
0,0,"Thompson, Myron H.",Democrat,male,African-American/black
1,1,"Shoob, Marvin A.",Democrat,male,white/caucasian
2,2,"Bua, Nicholas J.",Democrat,male,white/caucasian
3,3,"Kovachevich, Elizabeth",Republican,female,white/caucasian
4,4,"Gilliam, Earl B.",Independent/Other/Unknown,male,white/caucasian
...,...,...,...,...,...
993,548,"Nugent, Donald C.",Democrat,male,white/caucasian
994,549,"Sand, Leonard B.",Democrat,male,white/caucasian
995,550,"Weiner, Charles R.",Democrat,male,white/caucasian
997,551,"Karas, Kenneth M.",Republican,female,white/caucasian


In [81]:
#create table casedb_judge
run_query("""
  CREATE TABLE casedb_judge (
    judge_id INT NOT NULL,
    judge_name VARCHAR(255) NOT NULL,
    party_name VARCHAR(255) NOT NULL,
    gender_name VARCHAR(255) NOT NULL,
    race_name VARCHAR(255) NOT NULL,
    PRIMARY KEY (judge_id)
  );
""")
# run_query("SHOW TABLES")

ProgrammingError: 1050 (42S01): Table 'casedb_judge' already exists

In [82]:
#load data
data_tups = [tuple(x) for x in judge_df.values]
iq = "INSERT INTO casedb_judge (judge_id, judge_name, party_name, gender_name, race_name) VALUES (%s, %s, %s, %s, %s)"
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

In [83]:
#TEST
run_query("SELECT COUNT(*) FROM casedb_judge")

,COUNT(*)
0,553


In [84]:
#TEST
run_query('SELECT judge_name FROM casedb_judge WHERE judge_id = 2')

,judge_name
0,"Bua, Nicholas J."


## Make cases table. - 5 points

Finally we create the table that contains case's info: `casedb_cases`.

This table should have five columns and 1000 rows.

Note, one of these columns should be a judge_id that links to the judges table. You'll need to make this foreign key. You have two other foriegn_keys as well.



In [85]:
# select necessary columns to make cases_df
cases_df = all_cases_df[['case_id','casetype_id','category_id','case_year','judge_id']]

In [86]:
#create table casedb_cases (note that case_id requires larger data type than INT)
run_query("""
  CREATE TABLE casedb_cases (
    case_id BIGINT NOT NULL,
    casetype_id INT NOT NULL,
    year INT NOT NULL,
    direction_name VARCHAR(255) NOT NULL,
    judge_id INT NOT NULL,
    PRIMARY KEY (case_id))
""")


ProgrammingError: 1050 (42S01): Table 'casedb_cases' already exists

In [87]:
#load data into cases
data_tups = [tuple(int(item) if isinstance(item, np.int64) else item for item in x) for x in cases_df.to_numpy()]
iq = "INSERT INTO casedb_cases (case_id, casetype_id, year, direction_name, judge_id) VALUES (%s, %s, %s, %s, %s)"
conn, cur = get_conn_cur()
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

IntegrityError: 1062 (23000): Duplicate entry '28321332' for key 'casedb_cases.PRIMARY'

In [88]:
#TEST
run_query("SELECT COUNT(*) FROM casedb_cases;")

,COUNT(*)
0,1000


In [89]:
#TEST
run_query("SELECT * FROM casedb_cases WHERE case_id = 15660871")

,case_id,casetype_id,year,direction_name,judge_id
0,15660871,2,1,1983,2


## A quick test of your tables - 3 point

Below is a query to get the number of unique judges that have ruled on criminal court motion cases. You should get a value of 119 as your return if your database is set up correctly!

In [90]:
run_query("""SELECT COUNT(DISTINCT(casedb_judge.judge_id)) FROM casedb_cases
    JOIN casedb_judge ON casedb_cases.judge_id = casedb_judge.judge_id
        WHERE casetype_id = (SELECT casetype_id FROM casedb_casetype
                  WHERE casetype_name = 'criminal court motions'); """)


,COUNT(DISTINCT(casedb_judge.judge_id))
0,119


## Make rollup table - 7 points

Now let's make that rollup table! The goal here is to make a summary table easily accessed. We're going to roll the whole thing up by the judges party and the category, but you could imagine doing this for each judge to track how they make decisions over time which would then be useful for an analytics database. The one we're making could also be used as a dimension table where we needed overall party averages.

We want to get a percentage of liberal decisions by each grouping level (party_name, category_name). To do this we need first, the number of cases seen at each level, and second, the number of liberal decisions made at each level. `cases` contains the columns `libcon_id` which is a 0 if the decision was conservative in its ruling, and a 1 if it was liberal in its ruling. Thus, you can get a percentage of liberal decisions if you divide the sum of that column by the total observations. Your `agg()` can both get the sum and count.

After you groupby you'll need to reset the index, rename the columns, then make the percentage.

Once you do that you can push to a SQL table called 'rollup'

Let's get started

In [91]:
# Make a groupby called cases_rollup. This should group by party_name and categrory name. It should aggregate the count and sum of libcon_id
cases_rollup = all_cases_df.groupby(['party_name','category_name'])['libcon_id'].agg(['count','sum'])

In [92]:
# reset your index
cases_rollup.reset_index(inplace=True)

In [94]:
# rename your columns now. Keep the first to the same but call the last two 'total_cases' and 'num_lib_decisions'
cases_rollup = cases_rollup.rename(columns={'count': 'total_cases', 'sum': 'num_lib_decisions'})

Now make a new column called 'percent_liberal'

This should calucalte the percentage of decisions that were liberal in nature. Multiple it by 100 so that it's a full percent. Also use the `round()` function on the whole thing to keep it in whole percentages.

In [101]:
# make your metric called 'percent_liberal'
cases_rollup['percent_liberal'] = round(cases_rollup['num_lib_decisions'] / cases_rollup['total_cases'] * 100)
cases_rollup

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51.0
1,Democrat,Criminal Justice Cases,107,36,34.0
2,Democrat,Labor & Economic Cases,126,69,55.0
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42.0
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62.0
5,Independent/Other/Unknown,Labor & Economic Cases,13,6,46.0
6,Republican,Civil Liberties/Rights Cases,237,73,31.0
7,Republican,Criminal Justice Cases,125,34,27.0
8,Republican,Labor & Economic Cases,154,67,44.0



Now go and push the whole thing to a table called 'rollup'

There should be five columns and nine rows.

In [103]:
#create casedb_rollup table
run_query("""
  CREATE TABLE casedb_rollup (
    party_name VARCHAR(255) NOT NULL,
    category_name VARCHAR(255) NOT NULL,
    total_cases INT NOT NULL,
    num_lib_decisions INT NOT NULL,
    percent_liberal INT NOT NULL,
    PRIMARY KEY (party_name, category_name)
);
""")


""


In [107]:
#load data
data_tups = [tuple(x) for x in cases_rollup.to_numpy()]
iq = "INSERT INTO casedb_rollup (party_name, category_name, total_cases, num_lib_decisions, percent_liberal) VALUES (%s, %s, %s, %s, %s)"
conn, cur = get_conn_cur();
cur.executemany(iq, data_tups)
conn.commit()
conn.close()

In [108]:
# Run this cell
sql_head('casedb_rollup')

,party_name,category_name,total_cases,num_lib_decisions,percent_liberal
0,Democrat,Civil Liberties/Rights Cases,218,112,51
1,Democrat,Criminal Justice Cases,107,36,34
2,Democrat,Labor & Economic Cases,126,69,55
3,Independent/Other/Unknown,Civil Liberties/Rights Cases,12,5,42
4,Independent/Other/Unknown,Criminal Justice Cases,8,5,62
